In [ ]:
import torch
from torch.nn import Linear
from torch_geometric.nn import GCNConv

In [ ]:
class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(dataset.num_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = x.relu()

        embedding = x  # Save the output after the first layer

        x = self.conv2(x, edge_index)

        return embedding, x


In [ ]:
model = GCN(hidden_channels=16)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()
    for data in train_loader:  # Assuming you're using DataLoader
        data = data.to(device)
        optimizer.zero_grad()
        _, out = model(data)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()

def get_embeddings():
    model.eval()
    embeddings = []
    with torch.no_grad():
        for data in train_loader:
            data = data.to(device)
            embedding, _ = model(data)
            embeddings.append(embedding.cpu().numpy())
    return embeddings

for epoch in range(200):
    loss = train()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')